In [ ]:
import cv2
from jetbot import Camera, Robot
from http.server import BaseHTTPRequestHandler, HTTPServer
import threading
import urllib.parse
import logging
import json
import io
import torch
import time 

from algorithm import BoxFollower

In [ ]:
# Function to convert BGR8 images to JPEG
def bgr8_to_jpeg(value):
    return cv2.imencode('.jpg', value)[1].tobytes()

logger = logging.getLogger("RoboLog")
logging.basicConfig(filename="Robot.log", level=logging.INFO)

camera = Camera.instance(width=224, height=224)
robot = Robot()

running = True
paused = False
image = None

bf = BoxFollower()
box_thread = None
httpd = None

state = {}

In [ ]:
def box_follower_handler():
    global state, running, paused
    print("initialised")
    global bf
    while running:
        if not paused:
            image = camera.value
            decision, state = bf.step(image)
            logger.info(f"Decision: {state}")
            time.sleep(0.5)
    print("End")

def drive_with_turn_speed(forward_speed, turn_speed, wheel_base):
    """
    Set motor speeds based on forward speed and angular turn speed, capped to [-0.5, 0.5].

    Parameters:
    - forward_speed: float, desired linear speed (normalized, e.g., [-0.5, 0.5])
    - turn_speed: float, desired angular velocity in radians/sec (positive = left turn)
    - wheel_base: float, distance between the wheels (in meters)
    """
    # Compute differential drive wheel speeds
    left_speed = forward_speed - (turn_speed * wheel_base / 2)
    right_speed = forward_speed + (turn_speed * wheel_base / 2)

    # Normalize if either speed exceeds absolute cap of 0.5
    max_speed = max(abs(left_speed), abs(right_speed))
    if max_speed > 0.5:
        scale = 0.5 / max_speed
        left_speed *= scale
        right_speed *= scale

    # Clamp to [-0.5, 0.5]
    left_speed = max(min(left_speed, 0.5), -0.5)
    right_speed = max(min(right_speed, 0.5), -0.5)

    robot.set_motors(left_speed, right_speed)


# HTTP request handler class
class NetworkHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        try:
            global state, running, paused
            if self.path.startswith('/camera'):
                self.send_response(200)
                self.send_header('Content-type', 'image/jpeg')
                self.end_headers()
                self.wfile.write(bgr8_to_jpeg(camera.value))
            else:
                query = urllib.parse.urlparse(self.path).query
                params = urllib.parse.parse_qs(query)
                if self.path.startswith('/set_motors'):
                    left_speed = float(params.get('left', [0])[0])
                    right_speed = float(params.get('right', [0])[0])
                    robot.set_motors(left_speed, right_speed)
                    self.send_response(200)
                    self.end_headers()
                    self.wfile.write(b'Motors set')
                elif self.path.startswith('/set_drive'):
                    forward_speed = float(params.get('forward', [0])[0])
                    turn_speed = float(params.get('turn', [0])[0])
                    drive_with_turn_speed(forward_speed, turn_speed, 0.12)
                    self.send_response(200)
                    self.end_headers()
                    self.wfile.write(b'Drive set')
                elif self.path.startswith('/left'):
                    speed = float(params.get('speed', [0])[0])
                    robot.left(speed)
                    self.send_response(200)
                    self.end_headers()
                    self.wfile.write(b'Left command executed')
                elif self.path.startswith('/right'):
                    speed = float(params.get('speed', [0])[0])
                    robot.right(speed)
                    self.send_response(200)
                    self.end_headers()
                    self.wfile.write(b'Right command executed')
                elif self.path.startswith('/forward'):
                    speed = float(params.get('speed', [0])[0])
                    robot.forward(speed)
                    self.send_response(200)
                    self.end_headers()
                    self.wfile.write(b'Forward command executed')
                elif self.path.startswith('/stop'):
                    robot.stop()
                    self.send_response(200)
                    self.end_headers()
                    self.wfile.write(b'Stop command executed')
                elif self.path.startswith('/run_algo'):
                    paused = False
                    self.send_response(200)
                    self.end_headers()
                    self.wfile.write(b'Algorithm started')
                elif self.path.startswith('/stop_algo'):
                    paused = True
                    self.send_response(200)
                    self.end_headers()
                    self.wfile.write(b'Algorithm stopped')
                elif self.path.startswith('/get_weights'):
                    weights = bf.get_weights()
                    buffer = io.BytesIO()
                    torch.save(weights, buffer)
                    buffer.seek(0)
                    self.send_response(200)
                    self.send_header('Content-Type', 'application/octet-stream')
                    self.send_header('Content-Length', str(len(buffer.getvalue())))
                    self.end_headers()
                    self.wfile.write(buffer.read())

                elif self.path.startswith('/state'):
                    self.send_response(200)
                    self.send_header('Content-Type', 'application/json')
                    self.end_headers()
                    self.wfile.write(json.dumps(bf.decision).encode('utf-8'))
        except Exception as e:
            pass
                    

    def do_POST(self):
        if self.path.startswith('/set_weights'):
            content_length = int(self.headers.get('Content-Length', 0))
            if content_length > 0:
                raw_data = self.rfile.read(content_length)  # Read raw binary payload
                buffer = io.BytesIO(raw_data)
                state_dict = torch.load(buffer, map_location='cuda')
                # Set model weights
                bf.set_weights(state_dict)
                self.send_response(200)
                self.end_headers()
                self.wfile.write(b"Received and applied weights.")
            else:
                self.send_response(400)
                self.end_headers()
                self.wfile.write(b"No data received.")

def run_server():
    global box_thread, httpd

    box_thread = threading.Thread(target=box_follower_handler)
    box_thread.start()

    httpd = HTTPServer(('0.0.0.0', 8080), NetworkHandler)

    server_thread = threading.Thread(target=httpd.serve_forever)
    server_thread.start()
    print("Server started on port 8080.")

def stop_server():
    global running, httpd
    running = False
    if httpd:
        httpd.shutdown()
        httpd.server_close()
        httpd = None
    camera.stop()
    print("Server and box follower stopped cleanly.")

In [ ]:
run_server()

In [ ]:
stop_server()